# Seleção de features para criar dataframes ativo e inativo

O objetivo desse notebook é servir de base para analise de quais features são essenciais para definir quais usuários são ativos e quais são inativos.

Uma vez estabelecido esses critérios, ele poderá gerar um script para criar os dataframes de usuários ativos e inativos. Esses dataframes serão usados para a continuidade do processo de seleção e extração de features para posteriormente serem usados nas modelagens de cluster e churn.

No total vão ser 4 dataframes, 2 para usuários ativos e 2 para usuários inativos.
* inactiveUsersLatest.csv
* inactiveEvents_per_UsersLatest.csv
* activeUsersLatest.csv
* activeEvents_per_UsersLatest.csv

Os dataframes relacionados a usuários ativos serão usados para a modelagem de clusters e para a modelagem de churn.

Os dataframes relacionados a usuários inativos serão usados para a modelagem de churn.

Há bem mais usuários inativos do que usuários ativos. Assim para manter um balanceamento o mais próximo possível, foram selecioandas amostras de usuários dos mesmo cohorts que os usuários ativos. O intuito é que essa abordagem poderá ajudar na modelagem de churn.

Os dataframes de eventos (*xxxxxEvents_per_UsersLatest.csv*) se mantiveram os mesmos, foram apenas recortados para manterem eventos referentes aos usuários ativos ou inativos, conforme o caso.

Os dataframes de usuário foram extraídas as seguintes features:

* **firstAccess** - primeira data com evento registrado para o userId
* **lastAcess** - última data com evento registrado para o UserId
* **lasrDayAccess** - quantos dias o usuário acessou a plataforma pela última vez. A conta considera a data do último evento registrado menos a última data de acesso (lastAccess)
* **cohort** - usando o firstAccess, define um cohort para o usuário com base no ano-mês de entrada
* **donationLastXXDays** - usando o lastAccess são criadas features com a quantidade de dias em que o usuário acessou o app e fez um evento de doação em um determinado período. Será criado inicialmente duas columas. Uma para os últimos 120 dias e outra para os últimas 90 dias. Usaremos para fazer a definição de período ativo do usuário.
* **donationLastXXLabel** - usando o donationLastXXDays foram criados labels para fazer uma analise categórica desse campo. Os labels foram de acordo com a quantidade de doação, são eles: 1 dia ativo fazendo doação (Doou1), 2 dias ativos fazendo doação (Doou2), 3 dias ativos fazendo doação (Doou3), 3 a 8 dias ativos fazendo doações (Doou38), mais de 8 dias ativos fazendo doações. Nesse caso foram analisados dois períodos, 90 e 120 dias, assim temos duas colunas com labels.

Ainda para realizar uma analise de retenção é criado neste notebook um dataframe onde se agrupa os usuários de acordo com o mês em que realizaram atividades. Assim é apresentado alguns gráficos como: usuários únicos mensais em contraste com o tempo em que ele está na plataforma ou o mês em que ele entrou na plataforma (cohort).

### Iniciando as biblioteca

In [1]:
# importando os modulos principais

%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path

#from datetime import date, datetime
import matplotlib.pyplot as plt
#import os
import seaborn as sns
#import re
sns.set()

import locale
locale.setlocale(locale.LC_ALL, '')

'LC_CTYPE=pt_BR.UTF-8;LC_NUMERIC=pt_BR.UTF-8;LC_TIME=pt_BR.UTF-8;LC_COLLATE=en_US.UTF-8;LC_MONETARY=pt_BR.UTF-8;LC_MESSAGES=en_US.UTF-8;LC_PAPER=pt_BR.UTF-8;LC_NAME=pt_BR.UTF-8;LC_ADDRESS=pt_BR.UTF-8;LC_TELEPHONE=pt_BR.UTF-8;LC_MEASUREMENT=pt_BR.UTF-8;LC_IDENTIFICATION=pt_BR.UTF-8'

In [63]:
# Definindo os diretórios e arquivos
path_dados   = Path('/home/wesley/data/churn-prediction')
path_parquet = path_dados/'parquet'
path_csv     = path_dados/'csv'

file_parquet             = path_parquet/'flurry.parquet'
file_csv_users           = path_csv/'cp_users.csv'
file_csv_events          = path_csv/'cp_events.csv'
file_csv_events_per_user = path_csv/'cp_events_per_user.csv'
file_csv_users_features  = path_csv/'cp_users_features.csv'

In [3]:
%%time
# Carrega dataframe de eventos
events_per_user = pd.read_csv(file_csv_events_per_user, parse_dates=['eventDate'])
events_per_user.sort_values('eventDate', inplace=True)
print(events_per_user.shape)
events_per_user.info(),
events_per_user.head()

(3805376, 203)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3805376 entries, 0 to 3805375
Columns: 203 entries, eventDate to uncaught
dtypes: datetime64[ns](1), float64(201), int64(1)
memory usage: 5.8 GB
CPU times: user 38.5 s, sys: 13.2 s, total: 51.7 s
Wall time: 55.1 s


,eventDate,userId,ActivedNotificationIOS,ApparedMessageWrongLogin,AssociatedWithFacebook,ClickedAdAfterDonate,ClickedAdBeforeDonate,ClickedAdMoreInfo,ClickedBadgesList,ClickedBadgesUnupdated,...,rcdloi-ok,referrer,reportError,requestDeleteAccount,selectImpact,skipTutorialFromOnBoarding,test-referral-64,test-referral-70,test-referral-87,uncaught
0,2017-08-28,1111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,2017-10-02,123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,2017-10-02,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2017-10-02,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,2017-10-03,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Carrega dataframe de users
#users = pd.read_csv(path_csv/csv_usersFile, parse_dates=['firstAccess', 'lastAccess'])
users = pd.read_csv(file_csv_users)
print(users.shape)
users.info(),
users.head()

(210251, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210251 entries, 0 to 210250
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   userId       210251 non-null  int64 
 1   deviceModel  210251 non-null  object
 2   countryISO   210251 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.8+ MB


,userId,deviceModel,countryISO
0,1,Galaxy J5 Prime,BR
1,10,iOS Emulator,BR
2,100,Moto Z (2) Play,BR
3,1000,Nexus 5,BR
4,10000,Q6,BR


In [5]:
# carregando o data set de eventos
events = pd.read_csv(file_csv_events, parse_dates=['firstOccurrence', 'lastOccurrence'])
print(events.shape)
events.info(),
events.head()

(201, 37)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   eventName           201 non-null    object        
 1   firstOccurrence     201 non-null    datetime64[ns]
 2   lastOccurrence      201 non-null    datetime64[ns]
 3   intervalOccurrence  201 non-null    float64       
 4   2017-08             201 non-null    float64       
 5   2017-09             201 non-null    float64       
 6   2017-10             201 non-null    float64       
 7   2017-11             201 non-null    float64       
 8   2017-12             201 non-null    float64       
 9   2018-01             201 non-null    float64       
 10  2018-02             201 non-null    float64       
 11  2018-03             201 non-null    float64       
 12  2018-04             201 non-null    float64       
 13  2018-05             201 non-null    floa

,eventName,firstOccurrence,lastOccurrence,intervalOccurrence,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,...,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04
0,uncaught,2017-08-28,2020-04-16,962.0,2.0,0.0,89.0,220.0,164.0,39.0,...,17201.0,35093.0,33396.0,65754.0,111627.0,11675.0,3887.0,527.0,1308.0,1512.0
1,ativacao,2017-08-28,2017-08-28,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Opened,2017-10-02,2020-04-13,924.0,0.0,0.0,3811.0,5496.0,7167.0,8718.0,...,48.0,22.0,13.0,10.0,12.0,7.0,6.0,0.0,5.0,2.0
3,onBoardingStage,2017-10-02,2019-05-27,602.0,0.0,0.0,969.0,653.0,2547.0,3124.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,openCurtain,2017-10-02,2020-01-16,836.0,0.0,0.0,2262.0,2611.0,3513.0,3444.0,...,14.0,13.0,2.0,4.0,2.0,0.0,5.0,0.0,0.0,0.0


In [6]:
# Validando se está igual o número de usuários no dataframe de eventos e o de usuários
events_per_user.userId.nunique() == users.shape[0], events_per_user.userId.nunique(), users.shape[0]

(True, 210251, 210251)

In [7]:
# definindo duas matrizes uma com as colunas que são de eventos e a outra que não são
NO_EVENTS = ['userId', 'eventDate']
EVENTS = [c for c in events_per_user.columns if c not in NO_EVENTS]
len(EVENTS)

201

## Extraindo features básicas para o dataframe de usuários

Features:

* **firstAccess** - primeira data com evento registrado para o userId
* **lastAcess** - última data com evento registrado para o UserId
* **lastDayAccess** - quantos dias o usuário acessou a plataforma pela última vez. A conta considera a data do último evento registrado menos a última data de acesso (lastAccess)
* **cohort** - usando o firstAccess, define um cohort para o usuário com base no ano-mês de entrada
* **donationLastXXDays** - usando o lastAccess são criadas features com a quantidade de dias em que o usuário acessou o app e fez um evento de doação em um determinado período. Será criado inicialmente duas columas. Uma para os últimos 120 dias e outra para os últimas 90 dias. Usaremos para fazer a definição de período ativo do usuário.

In [8]:
# funções para aplicar no dataset 'users' para definir a linha de corte

# encontra o primeiro acesso do usuário
def funcFirstAccess(u):
    return events_per_user[events_per_user.userId == u['userId']]['eventDate'].iloc[0]

# encontra o último acesso do usuário
def funcLastAccess(u):
    return events_per_user[events_per_user.userId == u['userId']]['eventDate'].iloc[-1]

In [14]:
# calcula o número de dias entre o primeiro e último acesso do usuário
def funcCalendarDays(u):
    calendarDays = u['lastAccess'] - u['firstAccess'] + pd.DateOffset(1)
    return calendarDays.days

# encontra o número de dias que o usuário realizou algum evento no app
def funcActiveDays(u):
    return events_per_user[events_per_user.userId == u['userId']].shape[0]

In [10]:
# Aplica as funções para criar as features primeiro acesso, último acesso, total de doação
# Elas serão usadas para definir usuários ativos e inativos

# firstAccess - calcula o primeiro acesso dos usuários
%time users['firstAccess'] = users.apply(funcFirstAccess, axis=1)

# lastAccess - calcula o último acesso dos usuários
%time users['lastAccess'] = users.apply(funcLastAccess, axis=1)

%time users['cohort'] = users.firstAccess.map(lambda x: x.strftime('%Y-%m'))

CPU times: user 17min 10s, sys: 141 ms, total: 17min 11s
Wall time: 17min 11s
CPU times: user 16min 30s, sys: 347 ms, total: 16min 31s
Wall time: 16min 31s
CPU times: user 1.06 s, sys: 12 ms, total: 1.08 s
Wall time: 1.08 s


In [11]:
# cria a feature lastDayAccess que conta o número de dias que o usuário acessou a plataforma pela última vez
# considerando a última data de evento (lastDay). Usado para analise de cohort

lastDay = events_per_user.eventDate.sort_values().iloc[-1]  # encontra a última data do dataframe de eeventos
users['lastDayAccess'] = (lastDay - users.lastAccess).dt.days

In [15]:
# Aplica as funções para criar as features número de dias calendário, dias ativos e meia vida de dias ativos

# calendarDays - calcula o número de dias calendários na plataforma
%time users['calendarDays'] = users.apply(funcCalendarDays, axis=1)

# activeDays - calcula o número de dias ativos
%time users['activeDays'] = users.apply(funcActiveDays, axis=1)

CPU times: user 17 s, sys: 20 ms, total: 17.1 s
Wall time: 17.1 s
CPU times: user 15min 57s, sys: 39.8 ms, total: 15min 57s
Wall time: 15min 57s


In [18]:
events_per_user['Doou'] = events_per_user[['donate','Doou']].sum(axis=1)
events_per_user.drop('donate', axis=1, inplace=True)

In [22]:
# calcula o número de eventos de doação realizados pelo usuário
def funcDonationTotal(u):
    return events_per_user[events_per_user.userId == u['userId']]['Doou'].sum()

In [9]:
# donationAP - calcula os dias ativos fazendo doação no período de corte (AP)
#              o usuário deve ter feito uma ou mais doações para contabilizar
def funcTotalDonationAP(u):
    return events_per_user[(events_per_user.eventDate >= beginActivePeriod) &\
                (events_per_user.userId==u['userId']) & (events_per_user.Doou > 0)].shape[0]


In [47]:
# identifica a média de dias entre os eventos de doação
# caso o usuário tenha feito um ou nenhum evento de doação, retorno NaN
def funcAverageDaysBetweenDonationEvents(u):
    # cria um dataframe provisório com os eventos de doação do usuário
    dft = events_per_user[(events_per_user.userId == u['userId']) & (events_per_user.Doou>0)][['eventDate','Doou']]
    
    # verifica se existe 2 ou mais doações
    if dft.shape[0] > 1:
        # caso tenha mais de duas doações, vai calcular a diferença de dias
        first_time = True  # primeira vez que entra no loop, não calcula a diferença
        dif_total = 0      # acumulador da diferença entre dias de doação
        for i, e in reversed(list(enumerate(dft.iterrows()))):
            # se for a primeira vez, pega a última data para calcular a diferença
            if first_time:
                first_time = False
                ultimaData = e[1][0]
                continue
            # a partir do segundo evento, começa a totalizar a quantidade de dias entre os eventos
            novaData = e[1][0]
            dif = ultimaData - novaData
            dif_total += dif.days
            ultimaData = novaData
        # retorna a média de intervalo de dias entre os eventos de doação 
        return dif_total/(dft.shape[0]-1)
    else:
        # caso o usuário tenha feito uma ou nenhuma doação, retorna NaN
        return np.nan

In [23]:
# Aplica as funções para criar as features relacionadas a doação
# número de doações totais, media de eventos de doação por dias ativos, media de eventos de doação por dias calendários
# meia-vida dos eventos de doação

# donationTotal - calcula o total de eventos de doação por usuário
%time users['donationTotal'] = users.apply(funcDonationTotal, axis=1)

# averageEventActiveDay - calcula a média de doações por dias ativos
%time users['averageDonationEventActiveDay'] = users['donationTotal'] / users['activeDays']

# averageEventCalendarDay - calcula a média de doações por dias calendários
%time users['averageDonationEventCalendarDay'] = users['donationTotal'] / users['calendarDays']

CPU times: user 16min 48s, sys: 84.2 ms, total: 16min 48s
Wall time: 16min 50s
CPU times: user 617 µs, sys: 4 ms, total: 4.62 ms
Wall time: 12.7 ms
CPU times: user 1.72 ms, sys: 0 ns, total: 1.72 ms
Wall time: 1.47 ms


In [24]:
users.head()

,userId,deviceModel,countryISO,firstAccess,lastAccess,cohort,lastDayAccess,calendarDays,activeDays,donationLast90days,donationTotal,averageDonationEventActiveDay,averageDonationEventCalendarDay
0,1,Galaxy J5 Prime,BR,2017-11-07,2020-04-15,2017-11,1,891,75,14.0,175.0,2.333333,0.196409
1,10,iOS Emulator,BR,2017-10-18,2019-05-23,2017-10,329,583,21,NaN,5.0,0.238095,0.008576
2,100,Moto Z (2) Play,BR,2018-12-08,2018-12-08,2018-12,495,1,1,NaN,6.0,6.000000,6.000000
3,1000,Nexus 5,BR,2017-12-01,2017-12-01,2017-12,867,1,1,NaN,0.0,0.000000,0.000000
4,10000,Q6,BR,2018-06-24,2018-06-24,2018-06,662,1,1,NaN,0.0,0.000000,0.000000


In [48]:
# averageDaysBetweenDonationEvents - média de dias entre os eventos de doação
%time users['averageDaysBetweenDonationEvents'] = users.apply(funcAverageDaysBetweenDonationEvents, axis=1)

CPU times: user 39min 19s, sys: 164 ms, total: 39min 19s
Wall time: 39min 19s


In [49]:
# verificando a data do último evento e definindo a data de corte para definir usuários ativos
days_AP = 120 - 1                                           # define quantos dias é o AP - Active Period
beginActivePeriod = lastDay - pd.DateOffset(days=days_AP)   # define o dia em que inicia o AP

%time users['donationLast120days'] = users[users.lastAccess >= beginActivePeriod].\
apply(funcTotalDonationAP, axis=1)

CPU times: user 16min 50s, sys: 36 ms, total: 16min 50s
Wall time: 16min 50s


In [50]:
# verificando a data do último evento e definindo a data de corte para definir usuários ativos
days_AP = 90 - 1                                            # define quantos dias é o AP - Active Period
beginActivePeriod = lastDay - pd.DateOffset(days=days_AP)   # define o dia em que inicia o AP

%time users['donationLast90days'] = users[users.lastAccess >= beginActivePeriod].\
apply(funcTotalDonationAP, axis=1)

CPU times: user 14min 44s, sys: 99.9 ms, total: 14min 44s
Wall time: 14min 44s


In [51]:
# verificando a data do último evento e definindo a data de corte para definir usuários ativos
days_AP = 60 - 1                                            # define quantos dias é o AP - Active Period
beginActivePeriod = lastDay - pd.DateOffset(days=days_AP)   # define o dia em que inicia o AP

%time users['donationLast60days'] = users[users.lastAccess >= beginActivePeriod].\
apply(funcTotalDonationAP, axis=1)

CPU times: user 11min 41s, sys: 99.9 ms, total: 11min 41s
Wall time: 11min 41s


In [53]:
users.head()

,userId,deviceModel,countryISO,firstAccess,lastAccess,cohort,lastDayAccess,calendarDays,activeDays,donationLast90days,donationTotal,averageDonationEventActiveDay,averageDonationEventCalendarDay,averageDaysBetweenDonationEvents,donationLast120days,donationLast60days
0,1,Galaxy J5 Prime,BR,2017-11-07,2020-04-15,2017-11,1,891,75,14.0,175.0,2.333333,0.196409,3.888889,20.0,7.0
1,10,iOS Emulator,BR,2017-10-18,2019-05-23,2017-10,329,583,21,NaN,5.0,0.238095,0.008576,106.333333,NaN,NaN
2,100,Moto Z (2) Play,BR,2018-12-08,2018-12-08,2018-12,495,1,1,NaN,6.0,6.000000,6.000000,NaN,NaN,NaN
3,1000,Nexus 5,BR,2017-12-01,2017-12-01,2017-12,867,1,1,NaN,0.0,0.000000,0.000000,NaN,NaN,NaN
4,10000,Q6,BR,2018-06-24,2018-06-24,2018-06,662,1,1,NaN,0.0,0.000000,0.000000,NaN,NaN,NaN


In [54]:
users.columns

Index(['userId', 'deviceModel', 'countryISO', 'firstAccess', 'lastAccess',
       'cohort', 'lastDayAccess', 'calendarDays', 'activeDays',
       'donationLast90days', 'donationTotal', 'averageDonationEventActiveDay',
       'averageDonationEventCalendarDay', 'averageDaysBetweenDonationEvents',
       'donationLast120days', 'donationLast60days'],
      dtype='object')

In [64]:
users.to_csv(file_csv_users_features, index=False)

## Criação do dataframe de atividades mensais

Esse dataframe será bem importante para gerar alguns gráficos sobre retenção.

As observações são os meses do ano mais os usuários, a tupla (**month_event, userId**). O *eventMonth* foi criado a partir do *eventDate*. Como os usuários podem ter vários eventos em um mês. Foi removidos as chaves duplicadas de forma que para cada usuário tenha apenas um registro por mês, que é o suficiente para a analise a ser efetuada.

A features é a o tempo de vida do usuário na plataforma em dias no mês em que o evento foi realizado (**userRibonAge**). Usa-se como referência o *firstAccess* do usuário e o mês em que o evento está sendo realizado (*month_event*). Então, por exemplo, se o usuário 9999, que tem o firstAccess em 2018-01-01, se ele fez um evento no mês 2018-01, a idade dele é 1, no mês 2018-02, a idade é 31, no mês 2018-03, a idade é 61, e assim por diante. Se o usuário não tem registro de eventos no mês, não tem a observação do userId naquele mês.

Eventualmente poderia usar apenas o evento de doaçõa para ficar mais específico ainda. Inicialmente decidi não usar esse conceito.

In [55]:
# cria um dataframe apenas com o userId e eventDate
df = events_per_user[NO_EVENTS]

# cria uma coluna com o mês do evento. Terá vários eventos duplicados,
# pois um usuário em geral acessa o app várias vezes ao mês
%time df['month_event'] = df.eventDate.map(lambda x: x.strftime('%Y-%m'))

df.shape

CPU times: user 18.3 s, sys: 256 ms, total: 18.5 s
Wall time: 18.5 s


/home/wesley/anaconda3/envs/ribon/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(3805376, 3)

In [56]:
# verifica se inicialmente o tamanho dos dataframes são identicos
df.shape[0] == events_per_user.shape[0]

True

In [57]:
# cria um dataframe com um evento por usuário por mês
# ao mesmo tempo já exclui a coluna eventDate
df = df[['userId', 'month_event']].drop_duplicates()
df.shape, df.head()

((596629, 2),
    userId month_event
 0    1111     2017-08
 1     123     2017-10
 2       6     2017-10
 3       7     2017-10
 6       4     2017-10)

In [58]:
# verifica quantos usuários únicos por mês
df[['userId','month_event']].groupby('month_event').count().T

month_event,2017-08,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,...,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04
userId,1,354,419,678,820,548,587,1934,4750,5138,...,42952,50860,40658,46556,36406,31031,27234,27694,27885,23590


In [59]:
# funçào para verificar quantos dias o usuário estava na plataforma no mês em que o evento foi realizado
def funcDaysUsingPlat(r):
    month = pd.Timestamp(r['month_event'])
    user=r['userId']
    cohort=pd.Timestamp(users.loc[users.userId==user, 'firstAccess'].iloc[0].strftime('%Y-%m'))
    return (month - cohort).days

In [60]:
%%time
# criando a coluna usando a função
df['userRibonAge'] = df.apply(funcDaysUsingPlat, axis=1)

CPU times: user 5min 33s, sys: 68 ms, total: 5min 33s
Wall time: 5min 33s


In [62]:
df.sample(20)

,userId,month_event,userRibonAge
696977,81171,2019-05,0
1720312,72003,2019-08,92
951605,126055,2019-05,0
2984732,116262,2019-12,0
476353,36928,2019-03,59
3386510,209370,2020-02,62
1488848,15033,2019-07,334
2061931,174666,2019-08,0
3523762,247687,2020-03,29
387895,29611,2019-02,31
